In [68]:
import pickle
import numpy as np
import pandas as pd


data_dict = pickle.load(open('./data.pickle', 'rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])


def pad_or_trim(lst, target_length):
    return lst[:target_length] + [0]*(target_length - len(lst))

target_length = 42
#convert data format to df format
data_uniform = np.array([pad_or_trim(lst, target_length) for lst in data])

df = pd.DataFrame(data_uniform)

#add target column
df["target"]= labels
df["target"] = pd.to_numeric(df["target"])

C:\Users\HP\AppData\Local\Temp\ipykernel_3036\1240780232.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(data_dict['data'])


In [69]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,target
0,0.130155,0.405134,0.244957,0.328743,0.314981,0.196199,0.326993,0.085194,0.309596,0.0,...,0.297512,0.000000,0.156059,0.066890,0.144884,0.080069,0.230283,0.078499,0.282586,0
1,0.127657,0.406596,0.245127,0.336346,0.319178,0.199883,0.332182,0.083957,0.319657,0.0,...,0.301943,0.000000,0.140308,0.076361,0.154563,0.089709,0.242319,0.087678,0.297076,0
2,0.100242,0.316745,0.173345,0.260417,0.224636,0.155722,0.239240,0.068895,0.235638,0.0,...,0.210608,0.004798,0.131649,0.000000,0.096151,0.007162,0.152252,0.017402,0.198326,0
3,0.183669,0.487673,0.300495,0.386145,0.367937,0.233076,0.366365,0.105697,0.352769,0.0,...,0.357899,0.000000,0.216881,0.074491,0.184072,0.103190,0.280258,0.108597,0.347029,0
4,0.177842,0.489383,0.293759,0.394594,0.364016,0.232353,0.362253,0.102649,0.349510,0.0,...,0.364576,0.000000,0.218503,0.070585,0.187786,0.099163,0.284163,0.108559,0.352570,0


In [70]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
X = df.drop(columns=['target'])
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs'),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

#store evaluation metrics
results = []

for name, clf in classifiers.items():

    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    auc_roc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

    results.append({
        'Classifier': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'AUC ROC': auc_roc
    })
    print(f"Results saved for {name} model")

results_df = pd.DataFrame(results)

Results saved for Logistic Regression model
Results saved for Random Forest model
Results saved for Support Vector Machine model
Results saved for K-Nearest Neighbors model
Results saved for Decision Tree model


In [79]:
results_df

,Classifier,Accuracy,Precision,Recall,F1 Score,AUC ROC
0,Logistic Regression,0.965458,0.931214,0.930136,0.930356,0.991763
1,Random Forest,0.989060,0.988995,0.988452,0.988640,0.999579
2,Support Vector Machine,0.984939,0.950503,0.949902,0.950069,0.999705
3,K-Nearest Neighbors,0.988086,0.953111,0.953126,0.953077,0.997653
4,Decision Tree,0.971377,0.970379,0.970629,0.970432,0.984803
